In [ ]:
import sys
import pathlib

notebook_path = pathlib.Path().absolute()
# Add parent directory to path so we can import from PLD_accounting and Comparison
parent_dir = notebook_path.parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

import numpy as np
import matplotlib.pyplot as plt

from comparisons.data_management import load_results
from comparisons.experiments.epsilon_from_sigma import plot_epsilon_from_sigma, plot_epsilon_from_sigma_by_k, plot_epsilon_vs_k
from comparisons.experiments.PREAMBLE import plot_PREAMBLE_experiment
from comparisons.experiments.PREAMBLE import plot_preamble_epsilon_experiment
from comparisons.experiments.delta_comparison import plot_delta_comparison
from PLD_accounting.types import Direction
from comparisons.experiments.utility_comparison import plot_utility_comparison_results

# Fig. 1a: Epsilon from sigma (varying t, k=1)

In [ ]:
results = load_results('epsilon_from_sigma')
if results is None:
    print("ERROR: epsilon_from_sigma results not found!")
else:
    print(f"Loaded results with keys: {list(results.keys())}")
    if 'params' in results:
        print(f"Params: {list(results['params'].keys())}")
    for key in results.keys():
        if key != 'params':
            print(f"  {key}: {list(results[key].keys()) if isinstance(results[key], dict) else type(results[key])}")
    try:
        plot_epsilon_from_sigma(results)
    except Exception as e:
        print(f"ERROR plotting: {e}")
        import traceback
        traceback.print_exc()

# Fig. 1c: Epsilon from sigma across k (t=1000)


In [ ]:
results = load_results('epsilon_from_sigma_by_k')
if results is None:
    print("ERROR: epsilon_from_sigma_by_k results not found!")
else:
    print(f"Loaded results with keys: {list(results.keys())}")
    if 'params' in results:
        print(f"Params: {list(results['params'].keys())}")
    for key in results.keys():
        if key != 'params':
            print(f"  {key}: {list(results[key].keys()) if isinstance(results[key], dict) else type(results[key])}")
    try:
        plot_epsilon_from_sigma_by_k(results)
    except Exception as e:
        print(f"ERROR plotting: {e}")
        import traceback
        traceback.print_exc()


# Fig. 1d: Epsilon vs k (σ=1, t=1000)


In [ ]:
results = load_results('epsilon_vs_k')
if results is None:
    print("ERROR: epsilon_vs_k results not found!")
else:
    print(f"Loaded results with keys: {list(results.keys())}")
    if 'params' in results:
        print(f"Params: {list(results['params'].keys())}")
    for key in results.keys():
        if key != 'params':
            val = results[key]
            if isinstance(val, dict):
                print(f"  {key}: {list(val.keys())}")
            elif isinstance(val, list):
                print(f"  {key}: list(len={len(val)})")
            else:
                print(f"  {key}: {type(val)}")
    try:
        plot_epsilon_vs_k(results)
    except Exception as e:
        print(f"ERROR plotting: {e}")
        import traceback
        traceback.print_exc()


# PREAMBLE experiment

In [ ]:
results = load_results('PREAMBLE_experiment')
plot_PREAMBLE_experiment(results)

# Runtime and RDP Experiment

This experiment measures both the runtime and RDP accuracy of the geometric method.
- **Runtime plot**: Shows runtime vs. 1/loss_discretization for different t values
- **RDP plot**: Three subplots (one per t value) showing RDP vs. 1/loss_discretization with shared y-scale

Both plots use the same experimental parameters, making it easy to compare runtime cost with accuracy.

In [ ]:
from comparisons.experiments.runtime import plot_runtime_with_rdp_experiment

results = load_results('runtime_experiment')
if results is not None:
    print("="*70)
    print("Runtime and RDP Experiment Results")
    print("="*70)
    
    # Generate both runtime and RDP plots
    runtime_fig, rdp_fig = plot_runtime_with_rdp_experiment(
        results, 
        save_plots=False, 
        show_plots=True
    )
    
    print("\nGenerated two plots:")
    print("1. Runtime vs. 1/loss_discretization (single subplot)")
    print("2. RDP vs. 1/loss_discretization (three subplots with shared y-scale)")
else:
    print("Runtime experiment results not found. Please run the experiment first.")

# Delta Comparison Experiment

This experiment compares delta values for different methods across multiple parameter configurations.
Similar to the Chau et al. Monte Carlo comparison experiments.

In [ ]:
results = load_results('delta_comparison')
if results is not None:
    plot_delta_comparison(results, save_plots=False, show_plots=True)
else:
    print("Delta comparison results not found. Please run the experiment first.")

# Utility Comparison Experiment

In [ ]:
results = load_results('utility_comparison')
if results is None:
    print("Utility comparison results not found. Please run the experiment with RUN_UTILITY_COMPARISON=True")
else:
    plot_utility_comparison_results(results, show_ci=False)
    print(f"Plotted utility comparison for {len(results.get('experiments', []))} configurations")

# Epsilon Loss Discretization Experiment (σ=1)

This is the grid size experiment simplified to test only σ=1.  
Compares geometric vs FFT convolution methods across varying grid sizes.  
Shows epsilon-delta accuracy with runtime and memory profiling.

In [ ]:
# Import plotting function
from comparisons.experiments.grid_size import plot_grid_size_combined_all_directions

# Load epsilon loss_discretization results (sigma=1 only)
epsilon_disc_results = load_results('epsilon_discretization')

if epsilon_disc_results is not None:
    print("="*70)
    print("Epsilon Loss Discretization Experiment Results (σ=1)")
    print("="*70)
    print(f"\nAvailable result keys: {list(epsilon_disc_results.keys())}")

    # Plot epsilon results (delta=1e-6, sigma=1.0)
    print(f"\nPlotting ε for δ = 1e-06, σ = 1.0")
    fig, axs = plot_grid_size_combined_all_directions(
        epsilon_disc_results,
        delta=1e-6
    )
    plt.show()
else:
    print("Epsilon loss_discretization results not found.")
    print("Please run the experiment with RUN_EPSILON_DISCRETIZATION=True in paper_experiments.py")

# Subsampling Bounds Comparison

Compare subsampling via four competitors:
1. Analytic (Gaussian mixture ground truth)
2. dp_accounting (Gaussian mechanism with subsampling)
3. Our upper bound (subsample_PMF bounds)
4. Our lower bound (subsample_PMF bounds)

The plots show the CDF comparison and epsilon-ratio comparison.


In [ ]:
from comparisons.experiments.subsampling_comparison import plot_subsampling_comparison

subsampling_results = load_results("subsampling_comparison")
if subsampling_results is None:
    print("Subsampling comparison results not found. Please run run_subsampling_comparison first.")
else:
    print("="*70)
    print("Subsampling Bounds Comparison")
    print("="*70)
    params = subsampling_results['params']
    print(f"Parameters: σ={params['sigma']}, q={params['sampling_prob']}, discretization={params['discretization']}")
    plot_subsampling_comparison(subsampling_results, direction="remove", show_plots=True, save_plots=False)
    plot_subsampling_comparison(subsampling_results, direction="add", show_plots=True, save_plots=False)


# PREAMBLE epsilon

In [ ]:
results = load_results('preamble_epsilon_experiment')
plot_preamble_epsilon_experiment(results)